# COVID News Synthetic Data Generation - Structured Methodology

This notebook implements the 4-step methodology for synthetic data generation:

1. **Data Collection** (already done) - Real COVID news articles collected
2. **Fact Characterization** - Define fact types with name, description, and examples
3. **Fact Extraction** - Extract structured facts using LLM with predefined schema
4. **Fact Manipulation** - Modify facts and generate synthetic fake news articles

## Output Format
The pipeline generates JSON output with:
- `original_article` - Original news article
- `extracted_facts` - Structured facts with name, description, specific_data
- `modified_facts` - Modified facts before replacement
- `modified_article` - Final synthetic article

This follows the methodology for creating plausible but false COVID-related news content.

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import json
from datetime import datetime

# Import custom modules
from src.data_generation.llm_client import create_llm_client, SyntheticDataResult
from src.utils.data_utils import load_config, load_raw_data, save_processed_data
from src.utils.text_preprocessing import TextPreprocessor

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load configuration
config = load_config()
print("Configuration loaded:")
print(json.dumps(config, indent=2))

In [ ]:
# Load environment variables (API keys)
from dotenv import load_dotenv
load_dotenv()

# Check if API keys are available
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')

print(f"OpenAI API Key available: {openai_key is not None}")
print(f"Anthropic API Key available: {anthropic_key is not None}")

if not openai_key and not anthropic_key:
    print("\nWarning: No API keys found. Please set up your .env file with API keys.")
    print("Copy .env.example to .env and fill in your API keys.")

## Data Loading and Exploration

In [ ]:
# Load raw COVID news data
# Note: You'll need to add your COVID news dataset to data/raw/covid_news.csv
# Expected columns: 'title', 'content', 'date', 'source', etc.

try:
    news_df = load_raw_data("news")
    print(f"Loaded {len(news_df)} news articles")
    
    if len(news_df) > 0:
        print("\nDataset info:")
        print(news_df.info())
        print("\nFirst few rows:")
        print(news_df.head())
    else:
        print("No data found. Please add your COVID news dataset to data/raw/covid_news.csv")
        
        # Create sample data for demonstration
        sample_articles = [
            {
                'title': 'New COVID-19 Variant Detected in Major City',
                'content': 'Health officials in New York City announced today that a new variant of COVID-19 has been detected in 150 patients. The variant, designated B.1.1.7, appears to be 70% more transmissible than previous strains. Local hospitals have reported a 25% increase in admissions over the past week.',
                'source': 'Health News Daily',
                'date': '2024-01-15'
            },
            {
                'title': 'Vaccine Effectiveness Study Results Released',
                'content': 'A comprehensive study involving 50,000 participants shows that the latest COVID-19 vaccines maintain 95% effectiveness against severe illness. The research, conducted over 6 months, found that booster shots increase protection to 98% within two weeks of administration.',
                'source': 'Medical Research Journal',
                'date': '2024-01-10'
            }
        ]
        
        news_df = pd.DataFrame(sample_articles)
        print(f"\nCreated sample dataset with {len(news_df)} articles for demonstration.")
        
except Exception as e:
    print(f"Error loading data: {e}")
    news_df = pd.DataFrame()

## LLM Client Setup

In [ ]:
# Initialize LLM client
llm_provider = "openai"  # Change to "anthropic" if you prefer Claude

try:
    llm_client = create_llm_client(
        provider=llm_provider,
        model_name=config['llm'][llm_provider]['model'],
        temperature=config['llm'][llm_provider]['temperature'],
        max_tokens=config['llm'][llm_provider]['max_tokens']
    )
    print(f"Successfully initialized {llm_provider} client")
except Exception as e:
    print(f"Error initializing LLM client: {e}")
    print("Please check your API keys and configuration.")
    llm_client = None

In [ ]:
# Step 2: Fact Characterization - COVID-specific fact schema
# Following methodology: define facts with name, description, and common examples

from src.data_generation.fact_schemas import get_fact_schema, display_fact_schema, COVID_FACT_SCHEMA

# Load COVID-specific fact characterization schema
covid_fact_schema = get_fact_schema("news")

print("STEP 2: FACT CHARACTERIZATION")
print("="*40)
display_fact_schema(covid_fact_schema)

print(f"\nFact schema loaded with {len(covid_fact_schema)} fact types")
print("This schema will guide structured fact extraction in Step 3")

## Fact Extraction and Synthetic Data Generation

In [ ]:
# Step 3 & 4: Structured fact extraction and manipulation
def process_article_structured(article_text: str, llm_client, fact_schema: list) -> dict:
    """
    Process article using structured 4-step methodology:
    Step 3: Extract structured facts
    Step 4: Modify facts and generate synthetic article
    """
    try:
        print(f"  Step 3: Extracting structured facts...")
        # Extract structured facts using schema
        fact_result = llm_client.extract_structured_facts(article_text, fact_schema)
        
        if not fact_result.extracted_facts:
            print(f"  ⚠️  No facts extracted from article")
            return None
        
        print(f"  ✓ Extracted {len(fact_result.extracted_facts)} structured facts")
        
        print(f"  Step 4: Modifying facts...")
        # Modify facts to create false information
        modified_facts = llm_client.modify_facts(fact_result.extracted_facts)
        
        print(f"  ✓ Modified {len(modified_facts)} facts")
        
        print(f"  Step 4: Generating synthetic article...")
        # Generate synthetic article with modified facts
        synthetic_article = llm_client.generate_synthetic_article(article_text, modified_facts)
        
        print(f"  ✓ Generated synthetic article")
        
        # Return in required JSON format
        return {
            'original_article': article_text,
            'extracted_facts': fact_result.extracted_facts,
            'modified_facts': modified_facts,
            'modified_article': synthetic_article,
            'metadata': {
                'timestamp': datetime.now().isoformat(),
                'model': llm_client.model_name,
                'fact_schema_size': len(fact_schema)
            }
        }
    
    except Exception as e:
        print(f"  ❌ Error processing article: {e}")
        return None

print("Structured processing function defined (following 4-step methodology).")

In [ ]:
# Process articles using structured methodology (Steps 3 & 4)
if llm_client is not None and len(news_df) > 0:
    
    print("PROCESSING ARTICLES WITH STRUCTURED METHODOLOGY")
    print("="*50)
    
    synthetic_results = []
    
    # Start with 10 articles as per methodology (then 100, then full dataset)
    sample_size = min(10, len(news_df))
    
    print(f"Processing {sample_size} articles using 4-step methodology...")
    print("Following workflow: 10 articles → manual evaluation → 100 articles → full dataset")
    
    for idx, row in tqdm(news_df.head(sample_size).iterrows(), total=sample_size):
        # Use content column, or combine title and content
        if 'content' in row and pd.notna(row['content']):
            article_text = row['content']
        else:
            article_text = row['title'] if 'title' in row else str(row.iloc[0])
        
        print(f"\n📄 Processing article {idx + 1}/{sample_size}...")
        
        # Use structured processing function
        result = process_article_structured(article_text, llm_client, covid_fact_schema)
        
        if result:
            synthetic_results.append(result)
            print(f"  ✅ Successfully generated synthetic version")
        else:
            print(f"  ❌ Failed to generate synthetic version")
    
    print(f"\n🎉 COMPLETED: Generated {len(synthetic_results)} synthetic articles")
    print(f"Success rate: {len(synthetic_results)/sample_size*100:.1f}%")
    
else:
    print("⚠️  Skipping synthetic data generation due to missing LLM client or data.")
    synthetic_results = []

## Results Analysis and Visualization

In [ ]:
# Display structured results in required JSON format
if synthetic_results:
    print("STRUCTURED SYNTHETIC DATA RESULTS")
    print("="*50)
    print("Format: original_article, extracted_facts, modified_facts, modified_article")
    
    for i, result in enumerate(synthetic_results[:2]):  # Show first 2 in detail
        print(f"\n📰 ARTICLE {i + 1}")
        print("-"*30)
        
        print("\n? ORIGINAL ARTICLE:")
        print(f"  {result['original_article'][:200]}{'...' if len(result['original_article']) > 200 else ''}")
        
        print("\n🔹 EXTRACTED FACTS (Step 3):")
        for j, fact in enumerate(result['extracted_facts'], 1):
            print(f"  {j}. {fact.get('name_of_fact', 'Unknown')}")
            print(f"     Description: {fact.get('description_of_fact', 'N/A')}")
            print(f"     Specific Data: {fact.get('specific_data', 'N/A')}")
        
        print("\n? MODIFIED FACTS (Step 4 - before replacement):")
        for j, fact in enumerate(result['modified_facts'], 1):
            print(f"  {j}. {fact.get('name_of_fact', 'Unknown')}")
            print(f"     Modified Data: {fact.get('specific_data', 'N/A')}")
        
        print("\n? MODIFIED ARTICLE (Step 4 - final output):")
        print(f"  {result['modified_article'][:200]}{'...' if len(result['modified_article']) > 200 else ''}")
        
        print("\n" + "="*50)
    
    if len(synthetic_results) > 2:
        print(f"\n... and {len(synthetic_results) - 2} more results")
        
    print(f"\n📊 SUMMARY:")
    print(f"  - Total articles processed: {len(synthetic_results)}")
    total_facts = sum(len(r['extracted_facts']) for r in synthetic_results)
    print(f"  - Total facts extracted: {total_facts}")
    print(f"  - Average facts per article: {total_facts/len(synthetic_results):.1f}")
    
else:
    print("No synthetic results to display.")

In [ ]:
# Create summary statistics
if synthetic_results:
    # Create DataFrame for analysis
    analysis_data = []
    
    for result in synthetic_results:
        analysis_data.append({
            'original_length': len(result.original_text),
            'synthetic_length': len(result.synthetic_text),
            'original_word_count': len(result.original_text.split()),
            'synthetic_word_count': len(result.synthetic_text.split()),
            'facts_extracted': len(result.original_facts),
            'facts_modified': len(result.modified_facts)
        })
    
    analysis_df = pd.DataFrame(analysis_data)
    
    # Display summary statistics
    print("GENERATION STATISTICS")
    print("=" * 30)
    print(analysis_df.describe())
    
    # Visualizations
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('Synthetic Data Generation Analysis', fontsize=16)
    
    # Text length comparison
    axes[0, 0].scatter(analysis_df['original_length'], analysis_df['synthetic_length'])
    axes[0, 0].plot([0, analysis_df['original_length'].max()], [0, analysis_df['original_length'].max()], 'r--', alpha=0.5)
    axes[0, 0].set_xlabel('Original Text Length')
    axes[0, 0].set_ylabel('Synthetic Text Length')
    axes[0, 0].set_title('Text Length Comparison')
    
    # Word count comparison
    axes[0, 1].scatter(analysis_df['original_word_count'], analysis_df['synthetic_word_count'])
    axes[0, 1].plot([0, analysis_df['original_word_count'].max()], [0, analysis_df['original_word_count'].max()], 'r--', alpha=0.5)
    axes[0, 1].set_xlabel('Original Word Count')
    axes[0, 1].set_ylabel('Synthetic Word Count')
    axes[0, 1].set_title('Word Count Comparison')
    
    # Facts extracted distribution
    axes[1, 0].hist(analysis_df['facts_extracted'], bins=5, alpha=0.7, edgecolor='black')
    axes[1, 0].set_xlabel('Number of Facts Extracted')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Facts Extracted Distribution')
    
    # Facts modified distribution
    axes[1, 1].hist(analysis_df['facts_modified'], bins=5, alpha=0.7, edgecolor='black')
    axes[1, 1].set_xlabel('Number of Facts Modified')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Facts Modified Distribution')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("No data available for analysis.")

## Save Generated Data

In [ ]:
# Save synthetic data to files
if synthetic_results:
    # Create dataset with original and synthetic pairs
    synthetic_dataset = []
    
    for result in synthetic_results:
        # Add original (real) article
        synthetic_dataset.append({
            'text': result.original_text,
            'label': 'real',
            'type': 'original',
            'facts': '|'.join(result.original_facts),
            'generation_metadata': None
        })
        
        # Add synthetic (fake) article
        synthetic_dataset.append({
            'text': result.synthetic_text,
            'label': 'fake',
            'type': 'synthetic',
            'facts': '|'.join(result.modified_facts),
            'generation_metadata': json.dumps(result.generation_metadata)
        })
    
    # Convert to DataFrame and save
    synthetic_df = pd.DataFrame(synthetic_dataset)
    
    # Save to processed data folder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"synthetic_covid_news_{timestamp}.csv"
    
    save_processed_data(synthetic_df, filename, "synthetic")
    
    print(f"Saved {len(synthetic_df)} records to data/synthetic/{filename}")
    print(f"\nDataset breakdown:")
    print(synthetic_df['label'].value_counts())
    
    # Also save detailed results as JSON
    detailed_results = []
    for result in synthetic_results:
        detailed_results.append({
            'original_text': result.original_text,
            'original_facts': result.original_facts,
            'modified_facts': result.modified_facts,
            'synthetic_text': result.synthetic_text,
            'metadata': result.generation_metadata
        })
    
    json_filename = f"detailed_generation_results_{timestamp}.json"
    json_path = f"data/synthetic/{json_filename}"
    os.makedirs(os.path.dirname(json_path), exist_ok=True)
    
    with open(json_path, 'w') as f:
        json.dump(detailed_results, f, indent=2)
    
    print(f"Saved detailed results to {json_path}")
    
else:
    print("No synthetic data to save.")

## Next Steps

1. **Scale up**: Process larger datasets of COVID news articles
2. **Quality evaluation**: Manually review generated articles for quality
3. **Experiment with prompts**: Try different prompt engineering techniques
4. **Try different models**: Compare OpenAI vs Anthropic vs other LLMs
5. **Move to classification**: Use the generated data to train classification models

The generated synthetic data can now be used in the classification notebooks to train models that distinguish between real and fake news.